# auxiliar

In [42]:
import pandas as pd

In [43]:
!python -m spacy download es_core_news_sm

2023-07-06 02:28:41.913041: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 48.4 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [44]:
%pip install pycorrector
%pip install unidecode
%pip install pyspellchecker

In [45]:
tipificacion=pd.read_excel("/content/PRIORIZAR-1/data/retipi.xlsx").drop_duplicates()


tipificacion = tipificacion[tipificacion['correccion'] != 'conceptos facturados']

tipificacion=tipificacion.reset_index()


tipificacion


,index,NUMERO_PQR,DESCRIPCION_PQR_x,correccion
0,0,MDM-PQR-35424371,SG//LT: 6016133556 cuenta 12054410753// client...,INCREMENTO DE TARIFA
1,1,MDM-PQR-35424385,SG//LT: 6017257398 cuenta 12054364769// client...,INCREMENTO DE TARIFA
2,2,MDM-PQR-35424468,SG//LT: 6013835268 cuenta 12053820540// client...,INCREMENTO DE TARIFA
3,3,MDM-PQR-35425107,Línea: 6016456888 Nombre de quien se comunica:...,SVAS
4,4,MDM-PQR-35426399,CLIENTE DESEA CONOCER LA FACTURA GENERADA,otros
...,...,...,...,...
1330,1488,MDM-PQR-37429227,Nombre: LUIS ALFONSO RODRIGUEZ HERAZO CC: ...,compensacin
1331,1489,MDM-PQR-37429228,Nombre: GLORIA JANETH ALARCON ALARCON Cedula...,compensacin
1332,1490,MDM-PQR-37429229,clienta solicita informaci̟n referente a factu...,compensacin
1333,1491,MDM-PQR-37429230,NOMBRE:alejandra C.C. 79738807 TEL.3026604594...,compensacin


In [46]:
tipificacion['correccion'].value_counts()

INCREMENTO DE TARIFA      347
otros                     334
ASEGURAMIENTO             211
SVAS                      175
BENEFICIO DE RETENCION    154
compensacin               113
Name: correccion, dtype: int64

In [47]:
import pandas as pd

import gc

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification





nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_sm', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# aca empieza el modelo

In [48]:
sys.path.append('../librerias')


In [49]:
from clean_text import clean_text

In [50]:

#declare answers and questions
descripcion=tipificacion.DESCRIPCION_PQR_x	



# Cleaning the questions

clean_descripcion = []
for des in descripcion:
    clean_descripcion.append(clean_text(des))


#clean_descripcion=[k for k in clean_descripcion if 'soporte'  in k]

clean_descripcion  [881]

' unica porque retiro de hbo y paramot'

spelling

In [51]:
from spelling import correct_sentence


corrected_text_list = [correct_sentence(text) for text in clean_descripcion]
clean_descripcion=corrected_text_list

In [52]:
clean_descripcion[81]

'loca se penalizar asesor ya que no tiene encanta en el traslado paso de duo a trio por tanto el cobro de la clausula de permanencia en el servicio de tv'

lemma token 

In [53]:
from lemastem import data_preprocessing


clean_descripcion=data_preprocessing(clean_descripcion)
clean_descripcion[3]

['unicar', 'llego', 'cobro', 'hbo']

In [54]:
merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion'])


from unidecode import unidecode

def remove_accents(text):
    return unidecode(text)


# Aplicar la función a la columna 'Texto'
merged_df['descripcion'] = merged_df['descripcion'].apply(remove_accents)


tipificacion=pd.concat([tipificacion, merged_df], axis=1).loc[:, ['descripcion', 'correccion']]

#merged_df = pd.merge(merged_df,leo , on='index', how='inner').loc[:, ['descripcion', 'correccion']]
#############################################




tipificacion = tipificacion[tipificacion['correccion'] != 'SVAS']


tipificacion=tipificacion.dropna()

tipificacion

,descripcion,correccion
0,solicitar ajuste factura acuerdo cobro reconex...,INCREMENTO DE TARIFA
1,solicitar ajuste factura acuerdo cobro reconex...,INCREMENTO DE TARIFA
2,solicitar ajuste factura acuerdo cobro reconex...,INCREMENTO DE TARIFA
4,conocer factura generado,otros
6,locacion tarifa retencion ofrecer bajo oferta ...,ASEGURAMIENTO
...,...,...
1330,sensacion,compensacin
1331,unicar estar generar factura,compensacin
1332,sensacion fallo presento anterior,compensacin
1333,,compensacin


In [55]:
tipificacion.isnull().sum()

descripcion    0
correccion     0
dtype: int64

In [56]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Assuming you have a DataFrame called 'data' with 'categories' column
# where categories are labeled as 1, 2, and 3
mapping = {'otros': 0, 'INCREMENTO DE TARIFA': 1, 'ASEGURAMIENTO': 2,  'BENEFICIO DE RETENCION': 3,'compensacin': 4}

# Separate features and target variable
X = tipificacion.drop('correccion', axis=1)
y = tipificacion['correccion'].replace(mapping)

# Oversampling using RandomOverSampler
oversampler = RandomOverSampler(sampling_strategy='auto')
X_resampled, y_resampled = oversampler.fit_resample(X, y)






mapping = {'otros': 0, 'INCREMENTO DE TARIFA': 1, 'ASEGURAMIENTO': 2,  'BENEFICIO DE RETENCION': 3,'compensacin': 4}

crear matriz de numeros 

In [57]:
from tfidf import TFIDF

In [58]:

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from joblib import dump


X = X_resampled['descripcion']

y = y_resampled




# train test split (66% train - 33% test)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)




X_train,X_test = TFIDF(X_train,X_test)

pca = PCA()

pca.fit(X_train)

# Get the explained variance ratio for each component
explained_variance_ratio = pca.explained_variance_ratio_

# Calculate the cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

# Find the number of components that explain a desired amount of variance
desired_variance = 0.95  # Set the desired variance threshold
n_components = np.argmax(cumulative_variance_ratio >= desired_variance) + 1

# Create a new instance of PCA with the optimal number of components
pca = PCA(n_components=n_components)


X_train_new = pca.fit_transform(X_train)
X_test_new = pca.transform(X_test)

print("train with old features: ",np.array(X_train).shape)
print("train with new features:" ,np.array(X_train_new).shape)

print("test with old features: ",np.array(X_test).shape)
print("test with new features:" ,np.array(X_test_new).shape)





print('Training Data :', X_train.shape)

print('Testing Data : ', X_test.shape)


dump(value=pca, filename="/content/PRIORIZAR/modelo/pca_retipificacion.joblib")



tf-idf with 830 features
train with old features:  (1388, 830)
train with new features: (1388, 299)
test with old features:  (347, 830)
test with new features: (347, 299)
Training Data : (1388, 830)
Testing Data :  (347, 830)


['/content/PRIORIZAR-1/modelo/pca_retipificacion.joblib']

In [60]:

pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 7.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.2 MB/s eta 0:00:00


In [61]:
import optuna
from optuna.samplers import TPESampler

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score



In [62]:
def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 5,
        "n_estimators": trial.suggest_int("n_estimators", 20, 90),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 5.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 5.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        'learning_rate': trial.suggest_float("learning_rate", 0.04, 0.1),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    
    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train_new, y_train)
    preds = gbm.predict(X_test_new)
    accuracy = accuracy_score(y_test, preds)
    return accuracy

In [63]:
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="lightgbm", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20)


[I 2023-07-06 02:38:46,456] A new study created in memory with name: lightgbm


[LightGBM] [Warning] feature_fraction is set=0.48805353449026784, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.48805353449026784
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=0.010007110473405195, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.010007110473405195
[LightGBM] [Warning] bagging_fraction is set=0.4554031568612787, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4554031568612787
[LightGBM] [Warning] lambda_l1 is set=0.879294185249291, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.879294185249291


[I 2023-07-06 02:38:47,231] Trial 0 finished with value: 0.8501440922190202 and parameters: {'n_estimators': 49, 'lambda_l1': 0.879294185249291, 'lambda_l2': 0.010007110473405195, 'num_leaves': 29, 'feature_fraction': 0.48805353449026784, 'bagging_fraction': 0.4554031568612787, 'bagging_freq': 2, 'learning_rate': 0.060733643622582864, 'min_child_samples': 43}. Best is trial 0 with value: 0.8501440922190202.


[LightGBM] [Warning] feature_fraction is set=0.9268704618345672, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9268704618345672
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=0.7069470359901728, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7069470359901728
[LightGBM] [Warning] bagging_fraction is set=0.4164325559187557, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4164325559187557
[LightGBM] [Warning] lambda_l1 is set=0.1353297938831623, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1353297938831623


[I 2023-07-06 02:38:49,004] Trial 1 finished with value: 0.8501440922190202 and parameters: {'n_estimators': 58, 'lambda_l1': 0.1353297938831623, 'lambda_l2': 0.7069470359901728, 'num_leaves': 26, 'feature_fraction': 0.9268704618345672, 'bagging_fraction': 0.4164325559187557, 'bagging_freq': 5, 'learning_rate': 0.06503828814202763, 'min_child_samples': 58}. Best is trial 0 with value: 0.8501440922190202.


...

[I 2023-07-06 02:39:41,370] Trial 17 finished with value: 0.8703170028818443 and parameters: {'n_estimators': 40, 'lambda_l1': 0.016648647173036518, 'lambda_l2': 0.6750097622848942, 'num_leaves': 42, 'feature_fraction': 0.7146406171339541, 'bagging_fraction': 0.9157268507849452, 'bagging_freq': 6, 'learning_rate': 0.0753218952601706, 'min_child_samples': 21}. Best is trial 14 with value: 0.8847262247838616.


[LightGBM] [Warning] feature_fraction is set=0.8811471032245974, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8811471032245974
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=0.21099779135825067, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.21099779135825067
[LightGBM] [Warning] bagging_fraction is set=0.7784326966195179, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7784326966195179
[LightGBM] [Warning] lambda_l1 is set=0.02038096196411651, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.02038096196411651


[I 2023-07-06 02:39:43,944] Trial 18 finished with value: 0.8731988472622478 and parameters: {'n_estimators': 33, 'lambda_l1': 0.02038096196411651, 'lambda_l2': 0.21099779135825067, 'num_leaves': 20, 'feature_fraction': 0.8811471032245974, 'bagging_fraction': 0.7784326966195179, 'bagging_freq': 1, 'learning_rate': 0.08687674194094719, 'min_child_samples': 37}. Best is trial 14 with value: 0.8847262247838616.


[LightGBM] [Warning] feature_fraction is set=0.5881978503812422, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5881978503812422
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] lambda_l2 is set=1.5986778247844013, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.5986778247844013
[LightGBM] [Warning] bagging_fraction is set=0.9198759476179553, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9198759476179553
[LightGBM] [Warning] lambda_l1 is set=0.01004129209202005, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.01004129209202005


[I 2023-07-06 02:39:45,209] Trial 19 finished with value: 0.8645533141210374 and parameters: {'n_estimators': 46, 'lambda_l1': 0.01004129209202005, 'lambda_l2': 1.5986778247844013, 'num_leaves': 48, 'feature_fraction': 0.5881978503812422, 'bagging_fraction': 0.9198759476179553, 'bagging_freq': 6, 'learning_rate': 0.09407536572439132, 'min_child_samples': 99}. Best is trial 14 with value: 0.8847262247838616.


In [64]:
print('Best parameters:', study.best_params)

Best parameters: {'n_estimators': 41, 'lambda_l1': 0.049148437664035254, 'lambda_l2': 1.8266591621558763, 'num_leaves': 33, 'feature_fraction': 0.8256242530913764, 'bagging_fraction': 0.9030872285178855, 'bagging_freq': 3, 'learning_rate': 0.09934154462012913, 'min_child_samples': 23}


In [65]:
mode = lgb.LGBMClassifier(**study.best_params)
mode.fit(X_train_new, y_train)

[LightGBM] [Warning] lambda_l1 is set=0.049148437664035254, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.049148437664035254
[LightGBM] [Warning] bagging_fraction is set=0.9030872285178855, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9030872285178855
[LightGBM] [Warning] lambda_l2 is set=1.8266591621558763, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.8266591621558763
[LightGBM] [Warning] feature_fraction is set=0.8256242530913764, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8256242530913764
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


LGBMClassifier(bagging_fraction=0.9030872285178855, bagging_freq=3,
               feature_fraction=0.8256242530913764,
               lambda_l1=0.049148437664035254, lambda_l2=1.8266591621558763,
               learning_rate=0.09934154462012913, min_child_samples=23,
               n_estimators=41, num_leaves=33)

In [66]:
y_pred = mode.predict(X_test_new)


In [67]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, classification_report

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.83      0.88      0.86        67
           1       0.88      0.82      0.85        71
           2       0.93      0.95      0.94        65
           3       0.82      0.92      0.86        59
           4       0.96      0.87      0.91        85

    accuracy                           0.88       347
   macro avg       0.88      0.89      0.88       347
weighted avg       0.89      0.88      0.89       347



In [68]:
from joblib import dump


In [69]:
dump(value=mode, filename="/content/PRIORIZAR/modelo/retipificacion.joblib")

['/content/PRIORIZAR-1/modelo/retipificacion.joblib']